In [1]:
!pip install -q transformers datasets sentencepiece pymorphy2[fast]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.1/371.1 KB 37.4 MB/s eta 0:00:

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset, DatasetDict

In [3]:
model_name = 'ai-forever/ruT5-base'
dataset_name = 'artemsnegirev/ru-word-games'

In [4]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Prepare data

In [6]:
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/artemsnegirev___json/artemsnegirev--ru-word-games-3231cdc0217845fe/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


In [7]:
from collections import Counter
Counter(dataset["subset"])

Counter({'350_zagadok': 350,
         'bashnya_slov': 43522,
         'crosswords': 39290,
         'guess_answer': 1434,
         'ostrova': 1526,
         'top_seven': 6643,
         'ugadaj_slova': 7406,
         'umnyasha': 33052})

In [8]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def check_word_is_noun(w):
    p = morph.parse(w)[0]
    return p.tag.POS == 'NOUN'

In [9]:
subsets = ["350_zagadok", "ostrova", "ugadaj_slova", "umnyasha"]

dataset = dataset.filter(lambda x: x["subset"] in subsets)
dataset = dataset.filter(lambda x: check_word_is_noun(x["answer"]))
dataset = dataset.class_encode_column("subset")
dataset = dataset.train_test_split(test_size=0.1, stratify_by_column="subset")
dataset

Filter:   0%|          | 0/133223 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42334 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/25639 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/25639 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['subset', 'answer', 'prompt'],
        num_rows: 23075
    })
    test: Dataset({
        features: ['subset', 'answer', 'prompt'],
        num_rows: 2564
    })
})

In [10]:
prefix = 'guess word:'

def fix_ellipses(s):
    s = s.replace('....', '...')
    s = s.replace('(?)', '...')
    s = s.replace('…', '...')
    s = s.replace('...', ' ... ')

    return ' '.join(s.split())

def preprocess_data(examples):
    prompts = []

    for prompt in examples["prompt"]:
        prompt = fix_ellipses(prompt)
        prompt = prompt.replace('...', '<extra_id_0>')
        prompt = f'{prefix} {prompt}'
        prompts.append(prompt)

    labels = examples['answer']

    model_inputs = tokenizer(prompts, max_length=64, padding=True, truncation=True)
    labels = tokenizer(labels, max_length=8, padding=True, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [11]:
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/23075 [00:00<?, ? examples/s]

Map:   0%|          | 0/2564 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset.set_format(type='torch')

In [13]:
tokenized_dataset['train'][0]

{'subset': tensor(3),
 'answer': 'складка',
 'prompt': 'Прямолинейный сгиб на ткани',
 'input_ids': tensor([    8, 10434,  9539,     8, 25486,    23,  8772,    97,   640,    71,
            11, 15242,     9,  4417,     2,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([   11, 15276,     2,     0,     0,     0])}

# Teach model solve word puzzles

In [15]:
args = Seq2SeqTrainingArguments(
    "minibob",
    overwrite_output_dir=True,
    metric_for_best_model="em",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    weight_decay=0.1,
    warmup_ratio=0.01,
    num_train_epochs=8,
    save_total_limit=1,
    fp16=True,
    generation_max_length=8,
    predict_with_generate=True
)

In [16]:
import numpy as np


def exact_match(eval_pred):
    predictions, labels = eval_pred
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    hits, total = 0., 0
    for pred, label in zip(decoded_preds, decoded_labels):
        total += 1
        if pred.strip().lower() == label:
            hits += 1.

    return {'em': hits / total}

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=exact_match
)

In [18]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Em
1,1.963000,0.908287,0.154056
2,1.061200,0.867764,0.165367
3,0.892300,0.861685,0.175117
4,0.772300,0.862020,0.176677
5,0.687400,0.885043,0.186427
6,0.617300,0.897358,0.190328
7,0.568100,0.909559,0.195398
8,0.531500,0.919251,0.191108


TrainOutput(global_step=2888, training_loss=0.8866339443132818, metrics={'train_runtime': 1512.8542, 'train_samples_per_second': 122.021, 'train_steps_per_second': 1.909, 'total_flos': 1.071191183376384e+16, 'train_loss': 0.8866339443132818, 'epoch': 8.0})

# Try model in the wild

In [23]:
test_cases = [
{"answer": "год", "prompt": "этот период времени собирает 12 месяцев"},
{"answer": "человек", "prompt": "высшее млекопитающее"},
{"answer": "время", "prompt": "измеряется в часах, минутах и секундах"},
{"answer": "дело", "prompt": "может быть административное, уголовное ..."},
{"answer": "жизнь", "prompt": "противоположность смерти"},
{"answer": "день", "prompt": "время суток идущее после утра"},
{"answer": "рука", "prompt": "верхняя конечность человека"},
{"answer": "работа", "prompt": "место, куда человек устраивается после учебы"},
{"answer": "слово", "prompt": "составляющая предложений"},
{"answer": "место", "prompt": "формальное пространство чего либо"},
{"answer": "друг", "prompt": "близкий человек, не связанный романтическими чувствами"},
{"answer": "глаз", "prompt": "это орган с помощью которого человек может видеть"},
{"answer": "вопрос", "prompt": "когда нужно что-то узнать, надо задать ..."},
{"answer": "дом", "prompt": "это место в котором человек живет"},
{"answer": "сторона", "prompt": "обратная ... монеты"},
{"answer": "мир", "prompt": "после войны заключают ..."},
{"answer": "голова", "prompt": "часть тела, в которой находится мозг"},
{"answer": "ребенок", "prompt": "человек в малом возрасте"},
{"answer": "сила", "prompt": "физическое качество человека или животного"},
{"answer": "конец", "prompt": "есть начало, а есть ..."}
]

In [24]:
def predict_word(prompt):
    prompt = prompt.replace('...', '<extra_id_0>')
    prompt = f'{prefix} {prompt}'

    input_ids = tokenizer([prompt], return_tensors='pt').input_ids

    outputs = model.generate(
        input_ids.to(model.device), 
        num_beams=5, 
        max_new_tokens=8,
        do_sample=False,
        num_return_sequences=5
    )

    candidates = set()
        
    for tokens in outputs:
        candidate = tokenizer.decode(tokens, skip_special_tokens=True)
        candidate = candidate.strip().lower()

        candidates.add(candidate)

    return candidates

In [25]:
import json
import torch

model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    hits = 0

    print("failed predictions (true_answer - prompt - candidates)")

    for example in test_cases:
        answer = example['answer']
        prompt = example['prompt']

        candidates = predict_word(prompt)

        if answer in candidates:
            hits += 1
        else:
            print(answer, "-", prompt, "-", candidates)
    
    print("-" * 20)
    print("correct predictions:", hits / len(test_cases))

failed predictions (true_answer - prompt - candidates)
работа - место, куда человек устраивается после учебы - {'послед', 'отдых', 'отдаяние', 'откатка', 'интернат'}
слово - составляющая предложений - {'лексика', 'смысл', 'глагол', 'слог', 'слагаемое'}
место - формальное пространство чего либо - {'пространство', 'плоскость', 'форма', 'ткань', 'формат'}
друг - близкий человек, не связанный романтическими чувствами - {'сосун', 'неженка', 'согражданин', 'недруг', 'собрат'}
мир - после войны заключают ... - {'союз', 'контракт', 'пари', 'соглашение', 'договор'}
голова - часть тела, в которой находится мозг - {'левая сторона', 'миокард', 'энцефалит', 'мозг', 'спина'}
ребенок - человек в малом возрасте - {'малолеток', 'юнец', 'старик', 'малютка', 'малолетка'}
--------------------
correct predictions: 0.65


In [26]:
predict_word("это животное с копытами на нем ездят")

{'верблюд', 'конь', 'коня', 'лошадь', 'пони'}

# Upload model to 🤗 hub

In [27]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [29]:
model.push_to_hub("minibob")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/artemsnegirev/minibob/commit/861efd0f1aacdc1b978e8fe079967a2018279b5b', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='861efd0f1aacdc1b978e8fe079967a2018279b5b', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
tokenizer.push_to_hub("minibob")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/artemsnegirev/minibob/commit/832c721b06dd6abc4c8bfe04be070b167e9585b0', commit_message='Upload tokenizer', commit_description='', oid='832c721b06dd6abc4c8bfe04be070b167e9585b0', pr_url=None, pr_revision=None, pr_num=None)